In [3]:
! pip install fastai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 896.7 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 17.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 16.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.4 MB/s 

In [1]:
#IMPORT KAGGLE DATASET

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'nyc-taxi-trip-duration:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F6960%2F44258%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240220%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240220T163332Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D731c50cb1acdb246ccf9ef8740406a19426de0ff9b6a8c42e33c6eba218e971d4302fe78f141697ee10f87cb03deebe142505f1da56646de0863d19cc4b8f483171ecd9454f3ad484a26cbe465efe3f856c77cff621b74b775bf3191051b8c42e1ff3857a5407983fc23dd1698ecb06523117223db81a09ca51deab62266ed8ea16e2eae2c834139844692ff79e8c0adc662db917a6c7a5209a5c088df3b804ea760f5bdc2f85911587b5f3bbfbc6f18bfb7ac45d838fe2e33824399ea06136c610778ce6820a784e966d72d9be374fbc7bad44ccf5ef5be0453409d34112762dfb5c1dccaa972d95e044198750876cef3c20a91027e9571ab77d3fc2a140cf6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 89938099 bytes downloaded
Downloaded and uncompressed: nyc-taxi-trip-duration
Data source import complete.


In [2]:
#KAGGLE IMPORTS 
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings('ignore')



/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip
/kaggle/input/nyc-taxi-trip-duration/test.zip
/kaggle/input/nyc-taxi-trip-duration/train.zip


In [ ]:
#ADDITOIINAL IMPORTS

import fastai
from fastai.tabular.all import *

In [12]:
#SET GPU DEVICE
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
    device = torch.device('cuda')
else:
    print("CUDA is not available. CPU will be used.")
    device = torch.device('cpu')


print(torch.cuda.is_available())  # Should return True if CUDA is available and set up correctly
print(torch.cuda.current_device())  # Prints the current CUDA device index
print(torch.cuda.get_device_name(0))  # Prints the name of your CUDA device

CUDA is available. GPU will be used.


In [6]:
#GET DATAFRAME

df = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/train.zip', parse_dates=["pickup_datetime", "dropoff_datetime"])
# output the top of the data
df = df.drop('id', axis=1)
df = df.drop('store_and_fwd_flag', axis=1) #TODO(instead, change this to bool)
add_datepart(df, "pickup_datetime")
add_datepart(df, "dropoff_datetime")
df = df.drop('dropoff_datetimeElapsed', axis=1)
df = df.drop('pickup_datetimeElapsed', axis=1)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 31 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   vendor_id                         1458644 non-null  int64  
 1   passenger_count                   1458644 non-null  int64  
 2   pickup_longitude                  1458644 non-null  float64
 3   pickup_latitude                   1458644 non-null  float64
 4   dropoff_longitude                 1458644 non-null  float64
 5   dropoff_latitude                  1458644 non-null  float64
 6   trip_duration                     1458644 non-null  int64  
 7   pickup_datetimeYear               1458644 non-null  int32  
 8   pickup_datetimeMonth              1458644 non-null  int32  
 9   pickup_datetimeWeek               1458644 non-null  int32  
 10  pickup_datetimeDay                1458644 non-null  int32  
 11  pickup_datetimeDayofweek          145

In [24]:
#CREATE DATALOADERS


# Define your target variable and continuous feature names
# Step 2: Prepare your data
cat_names = ["vendor_id", 
              "pickup_datetimeYear", 
             "pickup_datetimeMonth", 
             "pickup_datetimeWeek", 
             "pickup_datetimeDay", 
             "pickup_datetimeDayofweek", 
             "pickup_datetimeDayofyear",
             "pickup_datetimeIs_month_end",
             "pickup_datetimeIs_month_start", 
             "pickup_datetimeIs_quarter_end", 
             "pickup_datetimeIs_quarter_start", 
             "pickup_datetimeIs_year_end",
             "pickup_datetimeIs_year_start",
             
             "dropoff_datetimeYear", 
             "dropoff_datetimeMonth", 
             "dropoff_datetimeWeek", 
             "dropoff_datetimeDay", 
             "dropoff_datetimeDayofweek", 
             "dropoff_datetimeDayofyear",
             "dropoff_datetimeIs_month_end",
             "dropoff_datetimeIs_month_start", 
             "dropoff_datetimeIs_quarter_end", 
             "dropoff_datetimeIs_quarter_start", 
             "dropoff_datetimeIs_year_end",
             "dropoff_datetimeIs_year_start"
            ]
#TODO: add year as categorical?
# TODO: or all things for pickup or dropoff are categorical?
batch_size = 40960
dls = TabularDataLoaders.from_df(df, y_names=["trip_duration"], cont_names=[col for col in df.columns if col not in cat_names], cat_names=cat_names, procs=[Categorify, Normalize], device=device, bs=batch_size)
dls.show_batch()

,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,2,2016,5,21,27,4,148,False,False,False,False,False,False,2016,5,21,27,4,148,False,False,False,False,False,False,1.0,-74.012390,40.709469,-74.011818,40.702541,412.0,412.0,412.0,412.0
1,1,2016,5,19,9,0,130,False,False,False,False,False,False,2016,5,19,9,0,130,False,False,False,False,False,False,1.0,-73.956131,40.767723,-73.958244,40.769653,163.0,163.0,163.0,163.0
2,1,2016,4,14,6,2,97,False,False,False,False,False,False,2016,4,14,6,2,97,False,False,False,False,False,False,1.0,-73.987556,40.719707,-73.980965,40.721115,342.0,342.0,342.0,342.0
3,1,2016,2,7,21,6,52,False,False,False,False,False,False,2016,2,7,21,6,52,False,False,False,False,False,False,1.0,-73.979202,40.755974,-73.921692,40.700649,1789.0,1789.0,1789.0,1789.0
4,1,2016,6,26,28,1,180,False,False,False,False,False,False,2016,6,26,28,1,180,False,False,False,False,False,False,1.0,-74.006165,40.740364,-74.006615,40.744125,149.0,149.0,149.0,149.0
5,1,2016,5,21,25,2,146,False,False,False,False,False,False,2016,5,21,25,2,146,False,False,False,False,False,False,1.0,-73.971443,40.760941,-73.993050,40.730900,1418.0,1418.0,1418.0,1418.0
6,1,2016,6,24,18,5,170,False,False,False,False,False,False,2016,6,24,19,6,171,False,False,False,False,False,False,1.0,-73.991310,40.723862,-74.008789,40.740601,743.0,743.0,743.0,743.0
7,2,2016,3,11,19,5,79,False,False,False,False,False,False,2016,3,11,19,5,79,False,False,False,False,False,False,3.0,-73.993492,40.752094,-74.002747,40.756187,280.0,280.0,280.0,280.0
8,1,2016,3,11,18,4,78,False,False,False,False,False,False,2016,3,11,18,4,78,False,False,False,False,False,False,1.0,-73.969681,40.752319,-73.954155,40.763931,345.0,345.0,345.0,345.0
9,1,2016,2,8,25,3,56,False,False,False,False,False,False,2016,2,8,25,3,56,False,False,False,False,False,False,1.0,-73.955551,40.776661,-73.973488,40.754799,406.0,406.0,406.0,406.0


In [25]:
#CREATE MODEL
learn = tabular_learner(dls, metrics=accuracy)

#See some info about model
print(f"{learn.loss_func = "})
print(f"{learn.lr = "})


epoch,train_loss,valid_loss,accuracy,time
0,18356010.000000,68194184.000000,0.000000,00:04
1,16078648.000000,68076312.000000,0.000000,00:04
2,17415508.000000,67986016.000000,0.000000,00:04
3,18749290.000000,67979200.000000,0.000000,00:05
4,18549628.000000,67852248.000000,0.000000,00:04
5,18454208.000000,67825928.000000,0.000000,00:04
6,19009166.000000,67826000.000000,0.000000,00:04
7,18046788.000000,67637656.000000,0.000000,00:04
8,17666540.000000,67623848.000000,0.000000,00:04
9,17995690.000000,67710872.000000,0.000000,00:04


In [43]:
#TRAIN 

nCycles = 50

learn.cuda()
learn.fit_one_cycle(nCycles)

epoch,train_loss,valid_loss,accuracy,time
0,14537938.000000,55381980.000000,0.000000,00:04
1,13677801.000000,53494536.000000,0.000000,00:04
2,13452965.000000,48278416.000000,0.000000,00:04
3,14142757.000000,59432844.000000,0.000000,00:04
4,14014529.000000,54010640.000000,0.000000,00:04
5,13238873.000000,50916852.000000,0.000000,00:04
6,13220232.000000,55965496.000000,0.000000,00:04
7,12489354.000000,47697424.000000,0.000000,00:04
8,11554410.000000,46257912.000000,0.000000,00:04
9,11189526.000000,52882028.000000,0.000000,00:04


In [44]:
#LOOK AT EXAMPLE PREDICTIONS

for i in range(10):
    row, clas, probs = learn.predict(df.iloc[i+30])

    row.show()

,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,5,20,17,1,138,False,False,False,False,False,False,2016,5,20,17,1,138,False,False,False,False,False,False,1.0,-73.97094,40.761669,-73.996239,40.725368,1479.0,1493.573364,1479.0,1493.573364


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,2,2016,5,20,21,5,142,False,False,False,False,False,False,2016,5,20,21,5,142,False,False,False,False,False,False,1.0,-73.998268,40.720039,-74.010361,40.672138,1048.0,1061.87561,1048.0,1061.87561


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,2,2016,5,19,10,1,131,False,False,False,False,False,False,2016,5,19,10,1,131,False,False,False,False,False,False,2.0,-74.005501,40.706589,-73.985817,40.744202,1022.0,1036.302979,1022.0,1036.302979


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,4,15,17,6,108,False,False,False,False,False,False,2016,4,15,17,6,108,False,False,False,False,False,False,1.0,-73.990311,40.75658,-73.870918,40.77393,2269.0,2274.751221,2269.0,2274.751221


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,1,1,9,5,9,False,False,False,False,False,False,2016,1,1,9,5,9,False,False,False,False,False,False,1.0,-74.008995,40.712589,-73.998634,40.722809,707.0,718.627075,707.0,718.627075


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,3,10,11,4,71,False,False,False,False,False,False,2016,3,10,11,4,71,False,False,False,False,False,False,1.0,-73.99456,40.750526,-73.9785,40.756191,526.0,537.601562,526.0,537.601562


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,1,1,5,1,5,False,False,False,False,False,False,2016,1,1,5,1,5,False,False,False,False,False,False,3.0,-73.951515,40.770248,-73.958527,40.761517,553.0,569.788513,553.0,569.788513


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,6,25,23,3,175,False,False,False,False,False,False,2016,6,25,23,3,175,False,False,False,False,False,False,1.0,-73.988068,40.728081,-73.980751,40.72147,277.0,296.939911,277.0,296.939911


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,5,21,27,4,148,False,False,False,False,False,False,2016,5,21,27,4,148,False,False,False,False,False,False,2.0,-73.978127,40.758343,-73.992249,40.730541,1189.0,1208.67688,1189.0,1208.67688


,vendor_id,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,dropoff_datetimeYear,dropoff_datetimeMonth,dropoff_datetimeWeek,dropoff_datetimeDay,dropoff_datetimeDayofweek,dropoff_datetimeDayofyear,dropoff_datetimeIs_month_end,dropoff_datetimeIs_month_start,dropoff_datetimeIs_quarter_end,dropoff_datetimeIs_quarter_start,dropoff_datetimeIs_year_end,dropoff_datetimeIs_year_start,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,trip_duration,trip_duration,trip_duration
0,1,2016,5,19,10,1,131,False,False,False,False,False,False,2016,5,19,10,1,131,False,False,False,False,False,False,1.0,-73.977768,40.774078,-73.947144,40.775993,1523.0,1528.067261,1523.0,1528.067261
